In [1]:
import requests
from pprint import pprint

# === CONFIG À ADAPTER SI BESOIN ===
API_BASE = "http://13.50.152.40:8080"  # ton EC2
USE_API_KEY = False                    # passe à True si tu protèges l'API
API_KEY = "CHANGE_ME_IF_NEEDED"

def make_headers():
    headers = {"Content-Type": "application/json"}
    if USE_API_KEY and API_KEY:
        headers["X-API-Key"] = API_KEY
    return headers

In [2]:
import json

def predict_tweet(text: str):
    """
    Envoie un tweet à l'API /predict et retourne
    la réponse JSON (dict) contenant prediction_id + timestamp.
    """
    url = f"{API_BASE}/predict"
    payload = {"text": text}
    
    resp = requests.post(url, headers=make_headers(), json=payload)
    try:
        data = resp.json()
    except Exception:
        print("Réponse brute :", resp.text)
        resp.raise_for_status()
    
    if resp.status_code != 200:
        print(f"[ERREUR] Status {resp.status_code}")
        pprint(data)
        raise RuntimeError("Erreur lors de l'appel /predict")
    
    return data

# Petit test (tu peux changer le texte)
example_pred = predict_tweet("I love Air Paradis!")
pprint(example_pred)

{'label': 'positive',
 'model_name': 'tweet_prediction',
 'model_stage': None,
 'model_version': '2',
 'raw_label': 'LABEL_1',
 'score': 0.9925695061683655}


In [4]:
def send_feedback(prediction_id: str, ground_truth: str):
    """
    Envoie le ground_truth ('positive' ou 'negative')
    pour une prédiction identifiée par prediction_id.
    """
    url = f"{API_BASE}/feedback"
    payload = {
        "prediction_id": prediction_id,
        "ground_truth": ground_truth
    }
    
    resp = requests.post(url, headers=make_headers(), json=payload)
    try:
        data = resp.json()
    except Exception:
        print("Réponse brute :", resp.text)
        resp.raise_for_status()
    
    if resp.status_code != 200:
        print(f"[ERREUR] Status {resp.status_code}")
        pprint(data)
        raise RuntimeError("Erreur lors de l'appel /feedback")
    
    return data

# Test rapide (⚠️ à utiliser après avoir obtenu un vrai prediction_id)
# send_feedback("un_prediction_id", "positive")

In [5]:
def interactive_one_tweet():
    text = input("✏️  Entrez un tweet (ou ligne vide pour arrêter) : ").strip()
    if not text:
        print("Fin.")
        return
    
    # 1) Prédiction
    pred = predict_tweet(text)
    prediction_id = pred.get("prediction_id")
    
    print("\n=== Prédiction du modèle ===")
    print(f"Texte       : {text}")
    print(f"Label       : {pred['label']} (score={pred['score']:.4f})")
    print(f"Modèle      : {pred['model_name']} v{pred['model_version']}")
    print(f"prediction_id : {prediction_id}")
    
    # 2) Validation humaine
    while True:
        ans = input("✅ La prédiction est correcte ? (o/n) : ").strip().lower()
        if ans in {"o", "n"}:
            break
        print("Merci de répondre par 'o' ou 'n'.")
    
    if ans == "o":
        ground_truth = pred["label"]  # même label que le modèle
    else:
        # On inverse
        if pred["label"] == "positive":
            ground_truth = "negative"
        elif pred["label"] == "negative":
            ground_truth = "positive"
        else:
            # cas bizarre, on demande à l'utilisateur
            ground_truth = input("👉 Entrez le vrai label ('positive' ou 'negative') : ").strip().lower()
    
    # 3) Envoi du feedback
    print(f"\nEnvoi du feedback : ground_truth = {ground_truth}")
    fb_res = send_feedback(prediction_id, ground_truth)
    print("Réponse /feedback :")
    pprint(fb_res)


# Lancer une validation pour un seul tweet
interactive_one_tweet()


=== Prédiction du modèle ===
Texte       : That cookie is delicious
Label       : positive (score=0.9962)
Modèle      : tweet_prediction v2
prediction_id : None

Envoi du feedback : ground_truth = positive
[ERREUR] Status 422
{'detail': [{'input': {'ground_truth': 'positive', 'prediction_id': None},
             'loc': ['body', 'timestamp'],
             'msg': 'Field required',
             'type': 'missing'}]}


RuntimeError: Erreur lors de l'appel /feedback